In [53]:
import re
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords, wordnet as wn
from collections import defaultdict
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import string
import pandas as pd

# Processing Data

In [54]:
df = pd.read_csv("movies.csv")

In [55]:
df.head(3)

,id,title,genres,original_language,overview,popularity,production_companies,release_date,budget,revenue,runtime,status,tagline,vote_average,vote_count,credits,keywords,poster_path,backdrop_path,recommendations
0,505642,Black Panther: Wakanda Forever,Action-Adventure-Science Fiction,en,Queen Ramonda Shuri M’Baku Okoye and the Dora ...,6117.070,Marvel Studios,2022-11-09,250000000.0,835000000.0,162.0,Released,Forever.,7.480,3040.0,Letitia Wright-Lupita Nyong'o-Danai Gurira-Win...,loss of loved one-hero-sequel-superhero-based ...,/sv1xJUazXeYqALzczSZ3O6nkH75.jpg,/xDMIl84Qo5Tsu62c9DGWhmPI67A.jpg,436270-829280-76600-56969-312634-1037858-238-5...
1,646389,Plane,Action-Adventure-Thriller,en,After a heroic job of successfully landing his...,4867.593,Di Bonaventura Pictures-MadRiver Pictures-Rive...,2023-01-13,25000000.0,29000000.0,107.0,Released,Survive together or die alone.,6.798,364.0,Gerard Butler-Mike Colter-Yoson An-Tony Goldwy...,philippines-held hostage-plane crash,/2g9ZBjUfF1X53EinykJqiBieUaO.jpg,/9Rq14Eyrf7Tu1xk0Pl7VcNbNh1n.jpg,937278-536554-653851-615777-9740-593643-555604...
2,315162,Puss in Boots: The Last Wish,Animation-Adventure-Comedy-Family-Fantasy,en,Puss in Boots discovers that his passion for a...,4607.945,Universal Pictures-DreamWorks Animation,2022-12-07,90000000.0,393000000.0,103.0,Released,Say hola to his little friends.,8.518,3598.0,Antonio Banderas-Salma Hayek-Harvey Guillén-Wa...,fairy tale-talking dog-spin off-aftercreditsst...,/kuf6dutpsT0vSVehic3EZIqkOBt.jpg,/tGwO4xcBjhXC0p5qlkw37TrH6S6.jpg,536554-76600-816904-653851-899112-985335-83078...


In [57]:
df = df.fillna(" ")
df.genres = df.genres.apply(lambda x: x.lower())

In [58]:
df.genres[0]

'action-adventure-science fiction'

Dans la celluce suivante on filtre pour obtenir un df avec les choix de genre de film de l'utilisateur

In [60]:
elements = ["science fiction", "adventure", "action"]
#df[df["genres"].str.contains("action")].head(3)
x = df
for e in elements:
    x = x[x["genres"].str.contains(e)]

In [61]:
def cleaning_func(data):
    data = data.replace("\n", "")
    data = re.sub("[\(\[].*?[\)\]]", "", data)                  # Here I remove les chiffres dans les chochets contained in wikipedia
    data = re.sub(r'[0-9]', '', data)                           # Here I remove all the numbers
    return data

In [62]:
def lemma_tokenize(data):
    words = word_tokenize(data)
    tag_map = defaultdict(lambda: wn.NOUN)
    tag_map['J'] = wn.ADJ
    tag_map['V'] = wn.VERB
    tag_map['R'] = wn.ADV

    lmtzr=WordNetLemmatizer()

    lemma_list=[]
    words = [x for x in words if x]

    for token, tag in nltk.pos_tag(words): #pos = position 
        lemma = lmtzr.lemmatize(token, tag_map[tag[0]])
        if lemma not in stopwords.words('english') and lemma not in string.punctuation:
            lemma_list.append(lemma)
    return lemma_list

In [63]:
def normalize(data):
    data_cleaned = cleaning_func(data)
    return lemma_tokenize(data_cleaned)

In [64]:
sentences = []
for e in x.overview:
    sentences.append(e)

Dans la cellule suivante la fonction utilise le nouveau df et prends en paramètre une mini description de l'utilisateur

In [65]:
# Argsort sorts an array and return an array of the sorted indexes
# Exemple l = [2,0,9,5]
# argsort(l) = [1,0,3,2]

def generateResponse(user_response):
    ai_res = ""
    sentences.append(user_response)                                     # I add the the user response in other to vectorize all of that
    tfidfvec = TfidfVectorizer(tokenizer = normalize)                   
    tfidf = tfidfvec.fit_transform(sentences)
    vals = cosine_similarity(tfidf[-1], tfidf[:-2])                     # This step is for calculating the similarity between the user response and the sentences containes in the text
    idx=vals.argsort()[0][-1]                                           # We take the most similar sentence.  the "[-1]" is because we want to take the highest indew (the most similar sentence)  
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-1]
    if(req_tfidf==0):
        return "I am sorry, I couldn't understand you"
    else:
        ai_res = ai_res + sentences[idx]
        sentences.remove(user_response)
        return ai_res

In [66]:
df[df.overview == generateResponse("love")]["title"]

117700    The Lawless Land
Name: title, dtype: object

In [68]:
df.iloc[117700].overview

'Action-adventure set in a world of the near-future where young love is forbidden among the lower classes. Nevertheless one couple defies their police-state world to fight and hold on to whatever love they can find.'